In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import utilities


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# ticker = "MSFT"
# df = utilities.load_stock(ticker)
ticker = 'SPY'
df = utilities.load_index(ticker)


In [ ]:
df = utilities.RSI(df, 14)
df = utilities.RSI(df, 28)
df = utilities.RSI(df, 56)
df = utilities.RSI(df, 100)
df = utilities.RSI(df, 200)

df = utilities.MACD(df)

df = utilities.EMA(df, 10)
df = utilities.EMA(df, 50)
df = utilities.EMA(df, 200)

df = utilities.market_hours_only(df)

df.head()

In [ ]:
# # Price normalizing

# days = utilities.list_of_day_dfs(df)
# days = [group.copy() for _, group in df.groupby(df['timestamp'].dt.date)]

# # for each day: sort the prices, then normalize them to 0-1 where 0 is the lowest price and 1 is the highest price for that day
# for day in days:
#     for col in ['high', 'low', 'volume', 'macd', 'signal', 'histogram', 'rsi14', 'rsi28', 'rsi56', 'ema10', 'ema50', 'ema200']:
#         day[col] = (day[col] - day[col].min()) / (day[col].max() - day[col].min())

# # recombine the days
# df = pd.concat(days)

In [ ]:
# Price normalizing needs to be the only metric that's forward looking

# Create a new column called 'enter' that represents whether or not the price will he higher or lower in $(some many) minutes
# 1 if the price will be higher, 0 if the price will be lower
df['enter'] = (df['close'].shift(-(60*7*5*4)) > df['close']).astype(int)
df = df.drop(columns=['open', 'close', 'timestamp', 'ticker', 'date', 'time'])

In [ ]:
df["enter"].value_counts()

In [ ]:
df["enter"].mean()

In [ ]:
# normalize all other columns
for col in df.columns:
    if col != 'enter':
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

In [ ]:
df.describe()

In [ ]:
# show the correlation matrix
correlation_matrix = df.corr()
plt.matshow(correlation_matrix)

plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90)
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)

plt.show()

In [ ]:
# train a basic neural network to predict the "enter" column

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['enter'])
y = df['enter']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=1, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
accuracy = model.evaluate(X_test_scaled, y_test)[1]
print(f"Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)

# Set display labels for binary classification
class_labels = ['0', '1']

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot()
plt.show()


Experiments: 

Failed / meaningless / accuracy within 50%:
- AAPL 5 mins
- SPY 5 mins

Possibly interesting:
- SPY 1 week: 59%? 
- SPY 2 week: 62%?
- SPY 4 week: 65%?
- AAPL 4 week: 63%

- TSLA 4 weeks: 55%